<img src="background.jpg" width="70%" >

## IBM Applied D.S Capstone Project
### Week 5 
## Content in Notebook

1. Import libraries
2. Scrap data from webpage into a DataFrame
    1. Data Preprocessing
    2. Output as Prediction file ( .csv)
3. Define Foursquare Credentials and Version
4. Top 225 venues that are within a radius of 600 meters for each post office
    1. Data Preprocessing
    2. Output as Prediction file ( .csv)
5. Analyze Each Postal Office For Venue Category
6. List and display the top 5 existing facilities for each Pin Code
7. Exploratory Visualization 1
8. Feature Engineering for Business Problem
   1. Simplification  
   2. Feature Selection  
   3. Handling Categorical Data (One Hot Encoding)
9. Potential area for the development of different infrastructure 
10. Best place to stay with vital infrastructure facilities nearby
11. Clustering And Exploratory Visualization 2
12. Examine Clusters
13. Observations:
14. Acknowledgments

## 1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd
import folium
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
#!conda install -c conda-forge lxml --yes
import requests
import lxml.html as lh
from sklearn.cluster import KMeans
print("Libraries imported.")

Libraries imported.


## 2. Checking & Reading the 'mumbailatlong' file directly to avoid data scraping.

In [7]:
clean_df = pd.read_csv(r'C:\Users\srinath.jettaboina\Desktop\Course Era\Project\Coordinated_Data.csv')
clean_df.head()

,City,Post Office,Pin Code,Latitude,Longitude
0,Hyderabad,Hyderabad G.P.O.,500001,17.8669,78.6003
1,Nampally,Gandhi Bhawan (Hyderabad),500001,17.8669,78.6003
2,Nampally,State Bank Of Hyderabad,500001,17.8669,78.6003
3,Nampally,Seetharampet,500001,17.8669,78.6003
4,Nampally,Moazzampura,500001,17.8669,78.6003


## OR Scrap data from webpage into a DataFrame

In [9]:
url='https://finkode.com/ap/hyderabad.html'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
r = requests.get(url)
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))
    size_of_col = 3
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 4, the //tr data is not from our table 
    if len(T)!=size_of_col:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df

1:"Post Office"
2:"District"
3:"Pincode"


,Post Office,District,Pincode
0,A.Gs Office S.O,Hyderabad,500004
1,A.Gs. Staff Quarters S.O,Hyderabad,500045
2,Administrative Buildings S.O,Hyderabad,500007
3,Aliabad B.O,Hyderabad,500078
4,Allembylines S.O,Hyderabad,500087
...,...,...,...
198,Yadgarpalli B.O,Hyderabad,501301
199,Yakutpura S.O,Hyderabad,500023
200,Yousufguda S.O,Hyderabad,500045
201,Zamistanpur S.O,Hyderabad,500020


In [11]:
clean_df["Latitude"] = ""
clean_df["Longitude"] = ""
df.shape
df.head()

,Post Office,District,Pincode
0,A.Gs Office S.O,Hyderabad,500004
1,A.Gs. Staff Quarters S.O,Hyderabad,500045
2,Administrative Buildings S.O,Hyderabad,500007
3,Aliabad B.O,Hyderabad,500078
4,Allembylines S.O,Hyderabad,500087


In [12]:
df1 = df.loc[0:25]
df2 = df.loc[26:50]
df3 = df.loc[51:75]
df4 = df.loc[76:100]
df5 = df.loc[101:125]
df6 = df.loc[126:150]
df7 = df.loc[151:156]

In [13]:
# Need to drop those Neighborhood that the geocode does not find
to_drop_unknown = []
geolocator = Nominatim(user_agent="ny_explorer")
for index, row in df1.iterrows():
    address = row['Post Office'] + ', Hyderabad'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df1.loc[index, 'Latitude'] = latitude
        df1.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

Cannot do: A.Gs Office S.O, Hyderabad, will drop index: 0
Cannot do: A.Gs. Staff Quarters S.O, Hyderabad, will drop index: 1
Cannot do: Administrative Buildings S.O, Hyderabad, will drop index: 2
The geograpical coordinate of Aliabad B.O, Hyderabad are 17.34563, 78.4726799.


c:\users\localadmin\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\localadmin\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Cannot do: Allembylines S.O, Hyderabad, will drop index: 4
The geograpical coordinate of Alwal S.O, Hyderabad are 17.5022292, 78.5088584.
Cannot do: Ambernagar B.O, Hyderabad, will drop index: 6
The geograpical coordinate of Amberpet S.O, Hyderabad are 17.390263150000003, 78.516481175.
Cannot do: Anandbagh S.O, Hyderabad, will drop index: 8
Cannot do: Anandnagar S.O (Hyderabad), Hyderabad, will drop index: 9
Cannot do: Anantaram B.O, Hyderabad, will drop index: 10
Cannot do: Andhra Mahila Sabha S.O, Hyderabad, will drop index: 11
Cannot do: Ankireddipalli B.O, Hyderabad, will drop index: 12
Cannot do: Ankushapur B.O, Hyderabad, will drop index: 13
Cannot do: Annajiguda B.O, Hyderabad, will drop index: 14
Cannot do: Aoc Records S.O, Hyderabad, will drop index: 15
Cannot do: AP Police Academy PO, Hyderabad, will drop index: 16
Cannot do: Aphb Colony Moulali S.O, Hyderabad, will drop index: 17
Cannot do: Ashoknagar S.O (Hyderabad), Hyderabad, will drop index: 18
Cannot do: Atvelli B.O, Hy

## 3. Finding the latitude and longitude of each zip code

In [28]:
df["Latitude"] = ""
df["Longitude"] = ""
df.shape

(157, 5)

In [29]:
df.head()

,City,Post Office,Pin Code,Latitude,Longitude
0,Mumbai,August Kranti Marg,400036,,
1,Mumbai,Aarey Milk Colony,400065,,
2,Mumbai,Andheri (East),400069,,
3,Mumbai,Andheri (West),400058,,
4,Mumbai,Antop Hill,400037,,


In [30]:
df1 = df.loc[0:25]
df2 = df.loc[26:50]
df3 = df.loc[51:75]
df4 = df.loc[76:100]
df5 = df.loc[101:125]
df6 = df.loc[126:150]
df7 = df.loc[151:156]

In [31]:
# Need to drop those Neighborhood that the geocode does not find
to_drop_unknown = []
geolocator = Nominatim(user_agent="ny_explorer")
for index, row in df1.iterrows():
    address = row['Post Office'] + ', Mumbai'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df1.loc[index, 'Latitude'] = latitude
        df1.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of August Kranti Marg, Mumbai are 18.9635489, 72.8099885.


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


The geograpical coordinate of Aarey Milk Colony, Mumbai are 19.1561292, 72.8707223.
The geograpical coordinate of Andheri (East), Mumbai are 19.1158835, 72.854202.
The geograpical coordinate of Andheri (West), Mumbai are 19.1172495, 72.833968.
The geograpical coordinate of Antop Hill, Mumbai are 19.0207608, 72.8652556.
Cannot do: Anu Shakti Nagar, Mumbai, will drop index: 5
Cannot do: B A R C, Mumbai, will drop index: 6
The geograpical coordinate of Ballard Estate, Mumbai are 18.9366512, 72.8391325.
The geograpical coordinate of Bandra (East), Mumbai are 19.0616565, 72.8498111.
The geograpical coordinate of Bandra (West), Mumbai are 19.0583358, 72.8302669.
The geograpical coordinate of Bangur Nagar, Mumbai are 19.1688142, 72.8336777.
The geograpical coordinate of Barve Nagar, Mumbai are 19.0952831, 72.9001776.
The geograpical coordinate of Bhandup, Mumbai are 19.1438684, 72.9384327.
The geograpical coordinate of Bhandup (East), Mumbai are 19.1485568, 72.9470657.
Cannot do: Bhavani Shan

In [32]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer2")
for index, row in df2.iterrows():
    address = row['Post Office'] + ', Mumbai'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df2.loc[index, 'Latitude'] = latitude
        df2.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Dadar, Mumbai are 19.019282, 72.8428757.
The geograpical coordinate of Dahisar, Mumbai are 19.2494501, 72.8596206.
The geograpical coordinate of Delisle Road, Mumbai are 18.9932602, 72.8310115.
The geograpical coordinate of Dharavi, Mumbai are 19.044463, 72.8586177.
The geograpical coordinate of F C I Mumbai, Mumbai are 18.9387711, 72.8353355.
The geograpical coordinate of Ghatkopar (West), Mumbai are 19.0897194, 72.9045972.
The geograpical coordinate of Girgaon, Mumbai are 18.9543165, 72.8179082.
The geograpical coordinate of Goregaon (East), Mumbai are 19.1692623, 72.8552548.
The geograpical coordinate of Goregaon (West), Mumbai are 19.1633281, 72.8411995.
The geograpical coordinate of Grant Road, Mumbai are 18.9644472, 72.8135727.
The geograpical coordinate of Hutatma Chowk, Mumbai are 18.9341699, 72.83252.
The geograpical coordinate of I I T Mumbai, Mumbai are 18.9387711, 72.8353355.
The geograpical coordinate of J B Nagar, Mumbai are 18.9387711, 72.83

In [14]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer3")
for index, row in df3.iterrows():
    address = row['Post Office'] + ', Hyderabad'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df3.loc[index, 'Latitude'] = latitude
        df3.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

Cannot do: Gandhi Bhawan S.O (Hyderabad), Hyderabad, will drop index: 51
The geograpical coordinate of Gandhinagar S.O (Hyderabad), Hyderabad are 17.4026882, 78.4673252.
Cannot do: Ghatkesar S.O, Hyderabad, will drop index: 53
Cannot do: Girmapur B.O, Hyderabad, will drop index: 54
Cannot do: Golconda Chowrastha S.O, Hyderabad, will drop index: 55
Cannot do: Golconda S.O, Hyderabad, will drop index: 56
Cannot do: Gowdavalli B.O, Hyderabad, will drop index: 57
Cannot do: Haiglines S.O, Hyderabad, will drop index: 58
Cannot do: Hakimpet S.O, Hyderabad, will drop index: 59
Cannot do: Hal S.O (Hyderabad), Hyderabad, will drop index: 60
The geograpical coordinate of Hanumanpet B.O, Hyderabad are 17.446169, 78.5265298.
The geograpical coordinate of High Court S.O (Hyderabad), Hyderabad are 17.3690184, 78.47172853081776.
Cannot do: Himayathnagar S.O, Hyderabad, will drop index: 63
Cannot do: Himmatnagar S.O, Hyderabad, will drop index: 64
The geograpical coordinate of HMT Township S.O, Hydera

In [15]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer4")
for index, row in df4.iterrows():
    address = row['Post Office'] + ', Hyderabad'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df4.loc[index, 'Latitude'] = latitude
        df4.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

Cannot do: Ie Moulali S.O, Hyderabad, will drop index: 76
The geograpical coordinate of IICT S.O, Hyderabad are 17.423913050000003, 78.53901458476338.
Cannot do: Jaggamguda B.O, Hyderabad, will drop index: 78
Cannot do: Jai Jawan Colony S.O, Hyderabad, will drop index: 79
Cannot do: Jama I Osmania S.O, Hyderabad, will drop index: 80
The geograpical coordinate of Jeedimetla B.O, Hyderabad are 17.5198207, 78.4427883.
Cannot do: JJ Nagar Colony S.O, Hyderabad, will drop index: 82
Cannot do: Jntu Kukat Pally S.O, Hyderabad, will drop index: 83
The geograpical coordinate of Jubilee Hills S.O, Hyderabad are 17.4308362, 78.4102882.
The geograpical coordinate of Kakatiya Nagar S.O, Hyderabad are 17.4146865, 78.5397653.
Cannot do: Kanchanbagh S.O, Hyderabad, will drop index: 86
The geograpical coordinate of Karwan Sahu S.O, Hyderabad are 17.3734894, 78.4412105.
Cannot do: Katchvanisingaram B.O, Hyderabad, will drop index: 88
Cannot do: Keesara B.O, Hyderabad, will drop index: 89
Cannot do: Kees

In [16]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer5")
for index, row in df5.iterrows():
    address = row['Post Office'] + ', Hyderabad'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df5.loc[index, 'Latitude'] = latitude
        df5.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Kushaiguda B.O, Hyderabad are 17.4825451, 78.5726444.
Cannot do: Kutbullapur B.O, Hyderabad, will drop index: 102
Cannot do: Kyasaram B.O, Hyderabad, will drop index: 103
The geograpical coordinate of L B Nagar S.O, Hyderabad are 17.349846, 78.5479412.
Cannot do: Lalgadi Malakpet B.O, Hyderabad, will drop index: 105
Cannot do: Lallaguda S.O, Hyderabad, will drop index: 106
Cannot do: Lallapet S.O, Hyderabad, will drop index: 107
Cannot do: LIC Division S.O, Hyderabad, will drop index: 108
Cannot do: Lunger House S.O, Hyderabad, will drop index: 109
The geograpical coordinate of Madhapur B.O, Hyderabad are 17.4408578, 78.3916289.
Cannot do: Malakpet Colony S.O, Hyderabad, will drop index: 111
Cannot do: Mamidipalli B.O, Hyderabad, will drop index: 112
The geograpical coordinate of Mangalhat S.O, Hyderabad are 17.3787687, 78.4604403.
Cannot do: Manovikasnagar S.O, Hyderabad, will drop index: 114
The geograpical coordinate of MG Road S.O, Hyderabad are 17.437

In [38]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer6")
for index, row in df6.iterrows():
    address = row['Post Office'] + ', Hyderabad'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df6.loc[index, 'Latitude'] = latitude
        df6.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

Cannot do: Napier Lines B.O, Hyderabad, will drop index: 126
The geograpical coordinate of Narayanguda S.O, Hyderabad are 17.3981159, 78.4885006.
Cannot do: Nehrunagar S.O (Hyderabad), Hyderabad, will drop index: 128
The geograpical coordinate of Neredmet S.O, Hyderabad are 17.470446, 78.5360054.
The geograpical coordinate of New Mla Quarters S.O, Hyderabad are 17.4052114, 78.4731670630351.
The geograpical coordinate of New Nallakunta S.O, Hyderabad are 17.3959752, 78.5092275.
The geograpical coordinate of Ngri S.O, Hyderabad are 17.4148264, 78.5463345.
The geograpical coordinate of Nimboliadda S.O, Hyderabad are 17.3845827, 78.4943227.
Cannot do: Nisa Hakimpet S.O, Hyderabad, will drop index: 134
Cannot do: Nutankal B.O, Hyderabad, will drop index: 135
The geograpical coordinate of Old Malakpet S.O, Hyderabad are 17.3787081, 78.4988983.
The geograpical coordinate of Old Mla Quarters S.O, Hyderabad are 17.3977019, 78.4823652.
The geograpical coordinate of Osmania General Hospital S.O, 

In [39]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer7")
for index, row in df7.iterrows():
    address = row['Post Office'] + ', Hyderabad'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df7.loc[index, 'Latitude'] = latitude
        df7.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

Cannot do: Raj Bhawan S.O (Hyderabad), Hyderabad, will drop index: 151
Cannot do: Rajbolaram B.O, Hyderabad, will drop index: 152
Cannot do: Ramakrishna Mutt S.O, Hyderabad, will drop index: 153
Cannot do: Ramakrishna Puram S.O, Hyderabad, will drop index: 154
Cannot do: Ramkoti S.O, Hyderabad, will drop index: 155
Cannot do: Rampallidiara B.O, Hyderabad, will drop index: 156


In [40]:
print(to_drop_unknown)

[0, 1, 2, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 25, 51, 53, 54, 55, 56, 57, 58, 59, 60, 63, 64, 66, 67, 68, 70, 72, 73, 74, 76, 78, 79, 80, 82, 83, 86, 88, 89, 90, 91, 93, 94, 95, 96, 97, 100, 102, 103, 105, 106, 107, 108, 109, 111, 112, 114, 117, 119, 120, 122, 125, 151, 152, 153, 154, 155, 156, 126, 128, 134, 135, 139, 140, 141, 142, 143, 145, 146, 147, 148, 149, 151, 152, 153, 154, 155, 156]


In [41]:
dfa = df1.append(df2, ignore_index = True)
dfa = dfa.append(df3, ignore_index = True)
dfa = dfa.append(df4, ignore_index = True)
dfa = dfa.append(df5, ignore_index = True)
dfa = dfa.append(df6, ignore_index = True)
dfa = dfa.append(df7, ignore_index = True)
cleandf = dfa.drop(to_drop_unknown)

In [42]:
cleandf.shape

(67, 5)

In [43]:
tosearch_df = df.loc[to_drop_unknown]
tosearch_df.head()
tosearch_df.reset_index(drop=True)

,Post Office,District,Pincode
0,A.Gs Office S.O,Hyderabad,500004
1,A.Gs. Staff Quarters S.O,Hyderabad,500045
2,Administrative Buildings S.O,Hyderabad,500007
3,Allembylines S.O,Hyderabad,500087
4,Ambernagar B.O,Hyderabad,500044
...,...,...,...
91,Rajbolaram B.O,Hyderabad,501401
92,Ramakrishna Mutt S.O,Hyderabad,500029
93,Ramakrishna Puram S.O,Hyderabad,500056
94,Ramkoti S.O,Hyderabad,500095


In [44]:
tosearch_df.shape

(96, 3)

In [45]:
tosearch_df1 = tosearch_df.iloc[0:25]
tosearch_df2 = tosearch_df.iloc[26:50]
tosearch_df3 = tosearch_df.iloc[51:55]

In [35]:
dfb = tosearch_df1.append(tosearch_df2, ignore_index = True)
dfb = dfb.append(tosearch_df3, ignore_index = True)
dfb

,Post Office,District,Pincode
0,A.Gs Office S.O,Hyderabad,500004
1,A.Gs. Staff Quarters S.O,Hyderabad,500045
2,Administrative Buildings S.O,Hyderabad,500007
3,Allembylines S.O,Hyderabad,500087
4,Ambernagar B.O,Hyderabad,500044
5,Anandbagh S.O,Hyderabad,500047
6,Anandnagar S.O (Hyderabad),Hyderabad,500004
7,Anantaram B.O,Hyderabad,500015
8,Andhra Mahila Sabha S.O,Hyderabad,500044
9,Ankireddipalli B.O,Hyderabad,501301


In [37]:
# define Foursquare Credentials and Version
CLIENT_ID = 'K03TPXVSGIZOO0SC0BJQNMHGLRIAPURVMXUVXPCGUNVZSZ2C' # your Foursquare ID
CLIENT_SECRET = 'SCFBMBY2RNLSCCUMC5LJAEUXY1EOO1OP1U0100LV23W4UEY4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
clean_df_new = clean_df.copy()

Your credentails:
CLIENT_ID: K03TPXVSGIZOO0SC0BJQNMHGLRIAPURVMXUVXPCGUNVZSZ2C
CLIENT_SECRET:SCFBMBY2RNLSCCUMC5LJAEUXY1EOO1OP1U0100LV23W4UEY4


# Checking & Reading the 'Hyderabadexplore' file directly to avoid api calling.

In [ ]:
radius = 600
LIMIT = 225
venues = []

for lat, long, pin, post, city in zip(clean_df_new['Latitude'], clean_df_new['Longitude'], clean_df_new['Pin Code'], clean_df_new['Post Office'], clean_df_new['City']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,        CLIENT_SECRET,        VERSION,        lat,        long,        radius,         LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post,            pin,            lat,             long,          city,
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
        venues_df = pd.DataFrame(venues)
        venues_df.head()

In [ ]:
# define the column names
venues_df.columns = ['Post Office', 'Pin Code', 'Latitude', 'Longitude', 'City', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

In [ ]:
venues_df.to_csv('Hyderabadexplore.csv') 

In [ ]:
venues_df.groupby(['Post Office', 'Pin Code', 'City']).count().head()

In [ ]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

In [ ]:
venues_df['VenueCategory'].unique()[:20]

# Analyze Each Postal Office For Venue Category

In [ ]:
# one hot encoding
Hyderabad_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
Hyderabad_onehot['Post Office'] = venues_df['Post Office'] 
Hyderabd_onehot['Pin Code'] = venues_df['Pin Code'] 
Hyderabad_onehot['City'] = venues_df['City'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(Hyderabad_onehot.columns[-3:]) + list(Hyderabad_onehot.columns[:-3])
Hyderabad_onehot = Hyderabad_onehot[fixed_columns]

print(Hyderabad_onehot.shape)
Hyderabad_onehot.head()

In [ ]:
Hyderabad_grouped = Hyderabad_onehot.groupby(["Post Office", "Pin Code", "City"]).mean().reset_index()

print(Hyderabad_grouped.shape)
Hyderabad_grouped.head()

# List and display the top 5 existing facilities for each Pin Code

In [ ]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ["Post Office", "Pin Code", "City"]
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Post Office'] = Hyderabad_grouped['Post Office']
neighborhoods_venues_sorted['Pin Code'] = Hyderabad_grouped['Pin Code']
neighborhoods_venues_sorted['City'] = Hyderabad_grouped['City']

for ind in np.arange(Hyderabad_grouped.shape[0]):
    row_categories = Hyderabad_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

In [ ]:
# Set manually to get proper fit in the map
address = 'Hyderabad'
latitude = 19.0760
longitude = 72.8777
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

# 7. Exploratory Visualization

In [ ]:
import folium
my_map = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, label1,common in zip(Hyderabad_merged['Latitude'], Hyderabad_merged['Longitude'], Hyderabad_merged['Post Office'],Hyderabad_merged['1st Most Common Venue'] ):
    labelnew =  'Post office : {} , Top Existing Infrastructure  : {}'.format(label1,common)
    label = folium.Popup( labelnew, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(my_map)  
my_map

# 8. Feature Engineering for Business Problem

In [51]:
# Quality Infrastructure 
search_query= ['Restaurant', 'Hotel', 'Farmers Market', 'Shopping Mall', 'Gym / Fitness Center', 'Pharmacy',
                         'Electronics Store', 'Indie Movie Theater', 'Light Rail Station','Metro Station', 'Train','Train Station', 'Garden',
                          'Theater','ATM', 'Office', 'Bus Station', 'Bank', 'Market' , 'Business Service', 'Monument / Landmark' ,
                          'Resort', 'Hospital', 'Police Station', 'School', 'College', 'Café' , 'Park', 'Playground',
                'Convention Center', 'College Auditorium', 'Government Building', 'Airport Terminal',
                         ]
print(len(search_query))

33


In [ ]:
quality_dataframe = []
quality_dataframe= venues_df.loc[venues_df['VenueCategory'].isin(search_query)]
quality_dataframe.shape

In [ ]:
# one hot encoding
qualityHyderabad_onehot = pd.get_dummies(quality_dataframe[['VenueCategory']], prefix="", prefix_sep="")
# add postal, borough and neighborhood column back to dataframe
qualityHyderabad_onehot['Post Office'] = quality_dataframe['Post Office'] 
qualityHyderabad_onehot['Pin Code'] = quality_dataframe['Pin Code'] 
qualityHyderabad_onehot['City'] = quality_dataframe['City'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(qualityHyderabad_onehot.columns[-3:]) + list(qualityHyderabad_onehot.columns[:-3])
qualityHyderabad_onehot = qualityHyderabad_onehot[fixed_columns]

print(qualityHyderabad_onehot.shape)
qualityHyderabad_onehot.head()
print(qualityHyderabad_onehot.columns.values)

In [ ]:
qualityHyderabad_grouped = qualitymumbai_onehot.groupby(["Post Office", "Pin Code", "City"]).sum().reset_index()
print(qualityHyderabad_grouped.shape)
qualityHyderabad_grouped.head()


# 13. Observations:

Most of the infrastructures are concentrated in the Southern areas of Hyderabad city, with the highest number in cluster 0 and moderate number in cluster 2. On the other hand, cluster 1 has a very low number of infrastructures in the neighborhoods. This represents a great opportunity and high potential areas to open new infrastructures as it is very little to no competition from existing varied infrastructures. Meanwhile, one can specifically check the infrastructure of choice against the postal office choice area.

A person who is planning to build infrastructure with unique selling propositions and lives prosperously to stand out from the competition can also open new infrastructures in neighborhoods in cluster 2 with moderate competition and supporting adequate no. of infrastructures. Lastly, people with planning to settle in the city are advised to start in cluster 0 which already has a high concentration of infrastructures.

# 14. Acknowledgement

In this project, I have gone through the process of identifying the
 business problems, specifying the data required, extracting and preparing
 the data, visualizing the results, performing machine learning by
 clustering the data into 3 clusters based on their frequency

 similarities, tackling and reaching to a definitive solution to 
business problems (mentioned in results). Lastly, the project is 
providing recommendations to the relevant stakeholders i.e. business 
developers regarding the best locations to open a new infrastructure.
 The project also provides visitors and immigrants to the city regarding
 postal office areas for growth and living prosperously.


 